In [0]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


## define useful fonctions

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, ActivityRegularization, Lambda
from keras.layers import Convolution2D, MaxPooling2D, Conv1D, MaxPooling1D, TimeDistributed
from keras.layers import AveragePooling2D, Input
from keras.utils import np_utils, normalize
from keras.engine import InputLayer
from keras import backend as K

import os
import scipy.io as sio
import h5py
import glob
import time

import sklearn
from sklearn.preprocessing import normalize

from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
from sklearn.linear_model import SGDClassifier

import numpy as np
import time
import math

RAM_SIZE = 4 * 1024 * 1024 * 1024 #Byte

def read_dir(dir_path) :
  """return the differents categories, the images in these categories, and the number of images per category"""
  listDir = sorted(os.listdir(dir_path))
  sizes = []
  listFiles = []
  for d in listDir :
    files = sorted(os.listdir(dir_path+'/'+d))
    sizes += [len(files)]
    listFiles += [files]
  return listDir, listFiles, sizes


def create_zero_array(layer, n, nb_images_left, nbImagesPerFile):
  """return a numpy array with zeros and with the needed scales"""
  nb = min(nb_images_left, nbImagesPerFile)
  if layer == 'block5_pool' :
    X = np.zeros((nb, 7 + 10*n, 7 + 10*n, 512))
  else :
    if n == 0 :
      X = np.zeros((nb, 4096))
    else :
      X = np.zeros((nb, 1 + 10*n, 1 + 10*n, 4096))
  return X
  

def complete_array(dir_path, listDir, listFiles, X, n, n_image) :
  """for all the images in listFiles, extract the features thanks to the network"""
  """return a 4-dimensional array with a 3-dimensional array per image"""
  X_new = X
  Y = []
  nb = 0
  t0 = time.time()
  cpt_X = 0
  for i in range (len(listDir)):
    d = listDir[i]
    files = listFiles[i]
    print(d, len(files))
    cpt = nb
    nb += len(files)
    if nb < n_image:
      continue
    for f in files:
      if cpt < n_image:
        cpt += 1
        continue
      if cpt_X == X.shape[0]:
        break
      t1 = time.time()
      img = image.load_img(dir_path+'/'+d+'/'+f, target_size=(224 + 320*n, 224 + 320*n))
      x = image.img_to_array(img)
      x = np.expand_dims(x, axis=0)
      x = preprocess_input(x)
      features = network2.predict(x)
      X_new[cpt_X,:] = features[0,:]
      Y.append(d)
      cpt += 1
      cpt_X += 1
      t2 = time.time()
      print('{}/{}: {}'.format(cpt, nb, t2-t1))
  t2 = time.time()
  print('total time: {}'.format(t2 - t0))
  return X_new, Y

def pool(X, p):
  """return the numpy array X on which the pooling p was performed"""
  if(len(X.shape) == 2):
    return X
  if p == 'max' :  
    X_pool = np.max(X, axis = (1, 2)) 
  elif p == 'mean' :
    X_pool = np.mean(X, axis = (1, 2)) 
  else :
    X_pool = X
  return X_pool


def to_fully_conv(model):
  """transforms the Convolutional Neural Network model into a Fully Convolutional Network"""

  new_model = Sequential()

  input_layer = InputLayer(input_shape=(None, None, 3), name="input_new")
#   input_layer = InputLayer(input_shape=(224+320*n, 224+320*n, 3), name="input_new")

  new_model.add(input_layer)

  for layer in model.layers:

    if "Flatten" in str(layer):
      flattened_ipt = True
      f_dim = layer.input_shape
#       print("get into flatten")
      continue

    elif "Dense" in str(layer):

      input_shape = layer.input_shape
      output_dim =  layer.get_weights()[1].shape[0]
      W,b = layer.get_weights()

      if flattened_ipt:
        shape = (f_dim[1],f_dim[2],f_dim[3],output_dim)
        new_W = W.reshape(shape)
        new_layer = Convolution2D(output_dim,
                                  (f_dim[1],f_dim[2]),
                                  strides=(1,1),
                                  activation=layer.activation,
                                  padding='valid',
                                  weights=[new_W,b])
        flattened_ipt = False

      else:
        shape = (1,1,input_shape[1],output_dim)
        new_W = W.reshape(shape)
        new_layer = Convolution2D(output_dim,
                                  (1,1),
                                  strides=(1,1),
                                  activation=layer.activation,
                                  padding='valid',
                                  weights=[new_W,b])

    else:
      new_layer = layer
    new_model.add(new_layer)

  return new_model
  

def cal_nbSaveFiles(layer, n, sizes_train, sizes_test):
  if layer == 'block5_pool':
    size_image = 8*(7+10*n)*(7+10*n)*512
  else:
    size_image = 8*(1+10*n)*(1+10*n)*4096
  nbSaveFiles_train = math.ceil(size_image * sum(sizes_train) / RAM_SIZE)
  nbSaveFiles_test = math.ceil(size_image * sum(sizes_test) / RAM_SIZE)
  nbImagesPerFile = int(RAM_SIZE // size_image)
  return nbSaveFiles_train, nbSaveFiles_test, nbImagesPerFile

print('Done')

Using TensorFlow backend.


Done


## Define global variables

In [0]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

data_path = 'drive/My Drive/medical_image_recognition/datasets/'
save_path = 'drive/My Drive/medical_image_recognition/notebooks_Chen/'

#layer from which we take the output : block5_pool, fc1 or fc2
layers = ['fc2']

#name of the document where the train and test set are stocked
docs = ['chest_xray']
scales = [2]
pooling = 'max'

## Calculer et sauvegarder les vecteurs

In [0]:
for doc in docs:
  #path where train images and test images are stocked
  dir_path_train = data_path + doc + '/train'
  dir_path_test =  data_path + doc + '/test'
  #creating numpy array from images
  listDir_train, listFiles_train, sizes_train = read_dir(dir_path_train)
  listDir_test, listFiles_test, sizes_test = read_dir(dir_path_test)
  for layer in layers:
    for n in scales:
      nbSaveFiles_train, nbSaveFiles_test, nbImagesPerFile = cal_nbSaveFiles(layer, n, sizes_train, sizes_test)
      save_train = []
      save_test = []
      save_train_y = []
      save_test_y = []
      for i in range(nbSaveFiles_train):
        #path where the output array will be saved
        save_train.append(save_path + doc + '/X_train_' + layer + '_' + pooling + '(' + str(n) + ')'+ str(i)+'.npy')
        save_train_y.append(save_path + doc + '/y_train_' + layer + '_' + pooling + '(' + str(n) + ')'+str(i)+'.npy')
      for i in range(nbSaveFiles_test):
        save_test.append(save_path + doc + '/X_test_' + layer + '_' + pooling + '(' + str(n) + ')'+ str(i)+'.npy')
        save_test_y.append(save_path + doc + '/y_test_' + layer + '_' + pooling + '(' + str(n) + ')'+str(i)+'.npy')

      print('dataset: {}, layer: {}, scale: {}'.format(doc, layer, n))
      #definition of the network used (VGG19)
      network = VGG19(weights='imagenet')#, input_shape=(fscale, fscale, 3))
      network2 = Model(inputs=network.input, outputs=network.get_layer(layer).output)

      if n >= 1 : 
        network2 = to_fully_conv(network2)

      for i in range(nbSaveFiles_train):
        X_train = create_zero_array(layer, n, sum(sizes_train)-i*nbImagesPerFile, nbImagesPerFile)

        X_train, Y_train = complete_array(dir_path_train, listDir_train, listFiles_train, X_train, n, i*nbImagesPerFile)
        X_train = pool(X_train, pooling)
        np.save(save_train_y[i], Y_train)
        np.save(save_train[i], X_train)
        del X_train
        print('{}th train file saved'.format(i))

      for i in range(nbSaveFiles_test):
        X_test = create_zero_array(layer, n, sum(sizes_test)-i*nbImagesPerFile, nbImagesPerFile)

        X_test, Y_test = complete_array(dir_path_test, listDir_test, listFiles_test, X_test, n, i*nbImagesPerFile)
        X_test = pool(X_test, pooling)
        np.save(save_test_y[i], Y_test)
        np.save(save_test[i], X_test)    
        del X_test
        print('{}th test file saved'.format(i))
          


dataset: chest_xray, layer: fc2, scale: 2
Instructions for updating:
Colocations handled automatically by placer.
NORMAL 1346
1/1346: 4.924823522567749
2/1346: 0.23940348625183105
3/1346: 0.23337817192077637
4/1346: 0.22338557243347168
5/1346: 0.22099566459655762
6/1346: 0.22878217697143555
7/1346: 0.2198786735534668
8/1346: 0.22066736221313477
9/1346: 0.22418522834777832
10/1346: 0.2270219326019287
11/1346: 0.22957491874694824
12/1346: 0.22769927978515625
13/1346: 0.22039580345153809
14/1346: 0.22779560089111328
15/1346: 0.24989962577819824
16/1346: 0.22260141372680664
17/1346: 0.2414994239807129
18/1346: 0.22311663627624512
19/1346: 0.22163057327270508
20/1346: 0.2245779037475586
21/1346: 0.23558282852172852
22/1346: 0.22324514389038086
23/1346: 0.21991991996765137
24/1346: 0.22812128067016602
25/1346: 0.2199709415435791
26/1346: 0.21767067909240723
27/1346: 0.23502874374389648
28/1346: 0.22383546829223633
29/1346: 0.2392430305480957
30/1346: 0.24568986892700195
31/1346: 0.2308704853

## SVM

In [0]:
#pooling performed on the output : max, mean, or not


for doc in docs:
  for layer in layers:
    for n in scales:
      #path where train images and test images are stocked
      dir_path_train = data_path + doc + '/train'
      dir_path_test =  data_path + doc + '/test'
      
      #creating numpy array from images
      listDir_train, listFiles_train, sizes_train = read_dir(dir_path_train)
      listDir_test, listFiles_test, sizes_test = read_dir(dir_path_test)
      nbSaveFiles_train, nbSaveFiles_test, nbImagesPerFile = cal_nbSaveFiles(layer, n, sizes_train, sizes_test)
      
      save_train = []
      save_test = []
      save_train_y = []
      save_test_y = []
      for i in range(nbSaveFiles_train):
        #path where the output array will be saved
        save_train.append(save_path + doc + '/X_train_' + layer + '_' + pooling + '(' + str(n) + ')'+ str(i)+'.npy')
        save_train_y.append(save_path + doc + '/y_train_' + layer + '_' + pooling + '(' + str(n) + ')'+str(i)+'.npy')
      for i in range(nbSaveFiles_test):
        save_test.append(save_path + doc + '/X_test_' + layer + '_' + pooling + '(' + str(n) + ')'+ str(i)+'.npy')
        save_test_y.append(save_path + doc + '/y_test_' + layer + '_' + pooling + '(' + str(n) + ')'+str(i)+'.npy')
      
      Y_train = np.load(save_train_y[0])
      Y_test = np.load(save_test_y[0])
      X_train = np.load(save_train[0])
      X_test = np.load(save_test[0])
      for i in range(1, nbSaveFiles_train):
        X_train = np.concatenate((X_train, np.load(save_train[i])))
        Y_train = np.concatenate((Y_train, np.load(save_train_y[i])))
      for i in range(1, nbSaveFiles_test):
        X_test = np.concatenate((X_test, np.load(save_test[i])))
        Y_test = np.concatenate((Y_test, np.load(save_test_y[i])))
      

      # normalization
      X_train = normalize(X_train, norm = 'l2', axis = 1)
      X_test = normalize(X_test, norm = 'l2', axis = 1)

      """SUPPORT VECTOR MACHINE"""

      parameters = {'C':[0.00001,0.0001,0.001,0.01, 0.1, 1, 10,100]}
      svr = svm.SVC(kernel='linear')
      clf = GridSearchCV(svr, parameters, cv=3)
      clf.fit(X_train, Y_train)
      
      print('test set Accuracy: %.2f' % clf.score(X_test, Y_test))
      print('best params: {}'.format(clf.best_params_))
      print('Done')

test set Accuracy: 0.80
best params: {'C': 10}
Done
test set Accuracy: 0.80
best params: {'C': 10}
Done


In [0]:
 nbSaveFiles_train